In [518]:
# importações
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep
from dotenv import load_dotenv
import os
import pandas as pd
import unicodedata

In [519]:
load_dotenv()

# carregando variáveis de ambiente
cpf = os.getenv('CPF')
senha = os.getenv('SENHA')

In [520]:
# carregando listas
PALAVRAS_SAUDE = ['medicina','fisioterapia','nutricao','farmacia', 'EDUCAÇÃO FÍSICA', 'biologicas', 'enfermagem', 'odontologia']
PALAVRAS_EXATAS = ['engenharia','matematica','estatistica', 'computacao', 'quimica', 'fisica', 'exatas','arquitetura']

In [521]:
# função para remocao de acentos e pontuação
def remover_acentos(texto):
    # Normaliza o texto decompondo caracteres acentuados em caracteres base e marcas de acento
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    
    # Filtra apenas os caracteres que não são marcas de acento
    return ''.join(c for c in texto_normalizado if not unicodedata.combining(c))


In [522]:
# digitando em qual linha da plianilha Distribuição de Demandas - CAT v2.xlsx começar a coletar

linha = int(input('Digite em qual linha da plianilha Distribuição de Demandas - CAT v2.xlsx a coleta deve começar'))
linha_inicio = linha - 3


In [523]:
# função para conexão do driver do selenium
def iniciar_driver():
    chrome_options = Options()

    arguments = ['--lang=pt-BR', '--start-maximized',
                '--incognito']

    for argument in arguments:
        chrome_options.add_argument(argument)


    driver = webdriver.Chrome(options=chrome_options)
    return driver

driver = iniciar_driver()


driver.get('https://siga.ufjf.br/')

In [524]:
sleep(3)

# Encontrar o iframe
frame = driver.find_element(By.XPATH,"//frameset/frame[@name='content']")

# Mudar para dentro do iframe
driver.switch_to.frame(frame)

# LOGANDO NO SIGA
# Encontrando o campo login
campo_login = driver.find_element(By.XPATH,"//html//body//div[@id='content-autenticacao']/div[3]/form/input[@type='text']")

sleep(1)

# preencher usuario no SIGA
campo_login.send_keys(cpf)

# selecionar e campo senha
sleep(1)

campo_senha = driver.find_element(By.XPATH,"//html//body//div[@id='content-autenticacao']/div[3]/form/input[@type='password']")
campo_senha.send_keys(senha)

# Encontrando o botão
botao_login = driver.find_element(By.XPATH,"//html//body//div[@id='content-autenticacao']/div[4]/div/a[@id='btnLogin3']/img[@class='botao'][1]")
botao_login.click()

driver.switch_to.default_content()

sleep(2)


In [525]:
# Encontrar o iframe
frame = driver.find_element(By.XPATH,"//frameset/frame[@name='content']")

# Mudar para dentro do iframe
driver.switch_to.frame(frame)

sleep(1)

# clicando em academico
driver.find_element(By.XPATH,"//div[@id='mainPanel']//div/a/div/span").click()

sleep(2)

# clicando em consultas
driver.find_element(By.XPATH,"//div[@class='mBoxPane']//div[2]/div/div//div[3]/a/div/span").click()

sleep(2)
# clicando em consulta alunos
driver.find_element(By.XPATH,"//div[@id='bodyLayout']/div/div/div//div[2]/div/div//div[2]/a/div/span").click()


# clicando no campo matricula
driver.find_element(By.XPATH,"//div/input[@name='matricula']").click()

In [526]:
# CRIANDO A TABELA COM OS NÚMEROS DE MATRÍCULA

# definindo as colunas da base que serão importadas
colunas = ['STATUS', 'Motivo para cancelamento', 'Matricula', 'CAMPUS', 'DATA PROTOCOLO', 'Nome completo:']

# importando a base
tabela = pd.read_excel('C:\projetos\Analise-de-cancelamentos-de-matricula-de-universidade\dados\Distribuição de Demandas - CAT v2.xlsx', sheet_name='Cancelamento de matrícula', usecols=colunas, header=1)


In [527]:
# eliminando linhas em branco

tabela = tabela.dropna(how='all')

# renomeando colunas
tabela_organizada = tabela.rename(columns={'STATUS':'Status','CAMPUS':'Campus','DATA PROTOCOLO': 'Data_protocolo', 
                                           'Nome completo:': 'Nome_completo', 'Motivo para cancelamento':'Motivo_cancelamento'})

# reordenando colunas
colunas = ['Status', 'Nome_completo', 'Motivo_cancelamento', 'Matricula', 'Campus', 'Data_protocolo']

tabela_organizada = tabela_organizada[colunas]

# selecionando a partir de qual linha deve-se considerar a tabela
tabela_organizada = tabela_organizada[linha_inicio:]

In [528]:
# ajustando o tipo de dados da coluna Data protocolo
pd.to_datetime(tabela_organizada['Data_protocolo'])
tabela_organizada['Data_protocolo'] = tabela_organizada['Data_protocolo'].dt.strftime('%d/%m/%Y')


In [529]:
# eliminando erros e sujeira nos números de matrícula
lista_matriculas = list(tabela_organizada['Matricula'])
lista_matriculas = [str(texto) for texto in lista_matriculas]
lista_matriculas = [matricula.replace(" ", "").replace("\t", "").replace("\n", "").replace("\r", "").replace("\xa0", "").strip() for matricula in lista_matriculas]
                        

In [530]:
# criação da nova base de dados após ralização do webscraping no SIGA

dados_alunos = []

contador = 0

print(f'Temos um total de {len(lista_matriculas)} alunos que solicitaram o cancelamento de matrícula')

for matricula in lista_matriculas:

    contador += 1

    print(f'Aluno {contador}/{len(lista_matriculas)}')


    # limpando o campo matrícula
    driver.find_element(By.XPATH,"//div/input[@name='matricula']").clear()

    sleep(1)

    driver.find_element(By.XPATH,"//div/input[@name='matricula']").send_keys(matricula)

    sleep(1)

    driver.find_element(By.XPATH,"//div[@id='formSituacaoAluno_row1']//span[@id='btnExibir_label']").click()

    sleep(1)


    # coletando os dados pessoais do aluno
    nome = driver.find_element(By.XPATH,"//div[@id='bgPessoais']//div/input").get_attribute('value')

    sleep(1)

    sexo = driver.find_element(By.XPATH,"//div[@id='bgPessoais_row4']/div/div/div/div[3]/input").get_attribute('value')

    sleep(1)

    data_nascimento = driver.find_element(By.XPATH,"//div[@id='bgPessoais_row4']/div/div[2]/div/div/input").get_attribute('value')

    sleep(1)


    # coletando os dados de ingresso do aluno
    curso = driver.find_element(By.XPATH,"//div[@id='bgIngresso']//div/input[@id='aluno::curso']").get_attribute('value')

    sleep(1)

    tipo_ingresso = driver.find_element(By.XPATH,"//div[@id='bgIngresso']//div[@id='bgIngresso_row3']//div/input[@id='programa::tipoIngresso']").get_attribute('value')

    sleep(1)

    habilitacao = driver.find_element(By.XPATH,"//div[@id='bgIngresso_row1']/div/div/div/div/input").get_attribute('value')

    sleep(1)

    pontuacao = driver.find_element(By.XPATH,"//div[@id='bgIngresso']//div[@id='bgIngresso_row3']//div/div[2]/div/div/input").get_attribute('value')

    sleep(1)

    classificacao = driver.find_element(By.XPATH,"//div[@id='bgIngresso_row3']//div/div[3]/div/div/input").get_attribute('value')

    sleep(1)

    semestre_ingresso = driver.find_element(By.XPATH,"//div[@id='bgIngresso_row2']//div[2]/div/div/input").get_attribute('value')

    ano_ingresso = matricula[:4]


    # tratamento e preenchimento area

    curso_tratado = remover_acentos(curso.split(' - ')[1].lower())

    if any(palavra in curso_tratado for palavra in PALAVRAS_SAUDE):
        area = 'Saúde'

    elif curso_tratado == "educacao fisica":
        area = 'Saúde'

    elif curso_tratado == "ead":
        area = 'Não determinado'

    elif any(palavra in curso_tratado for palavra in PALAVRAS_EXATAS):
        area = 'Exatas'

    else:
        area = 'Humanas'



    # Adicionando os dados do aluno à lista

    dados_alunos.append({
        "Matricula": matricula,
        "Nome": nome,
        "Sexo": sexo,
        "Data_nascimento": data_nascimento,
        "Curso": curso,
        "Area": area,
        "Tipo_ingresso": tipo_ingresso,
        "Habilitacao": habilitacao,
        "Pontuacao": pontuacao,
        "Classificacao": classificacao,
        "Semestre_ingresso": semestre_ingresso,
        "Ano_ingresso": ano_ingresso
    })

# Criando um DataFrame com os dados
df_alunos = pd.DataFrame(dados_alunos)


print('Coleta de dados terminada!')

Temos um total de 6 alunos que solicitaram o cancelamento de matrícula
Aluno 1/6
Aluno 2/6
Aluno 3/6
Aluno 4/6
Aluno 5/6
Aluno 6/6
Coleta de dados terminada!


In [531]:
# resetando o índice da tabela_organizada
tabela_organizada.reset_index(inplace=True)

# excluindo a coluna Matrícula da tabela df_alunos
df_alunos.drop(columns='Matricula', inplace = True)  

# excluindo a coluna index da tabela df_alunos
tabela_organizada.drop(columns='index', inplace=True)  

# unindo as duas tabelas
tabela_completa = tabela_organizada.join(df_alunos, how='inner')


In [532]:
# reordenando colunas
colunas = ['Status', 'Data_protocolo', 'Matricula', 'Nome_completo', 'Sexo',
       'Data_nascimento', 'Campus', 'Motivo_cancelamento', 'Curso',
       'Area', 'Habilitacao', 'Tipo_ingresso', 'Pontuacao', 'Classificacao',
       'Ano_ingresso', 'Semestre_ingresso']

tabela_completa = tabela_completa[colunas]

In [533]:
# exportando a tabela
tabela_completa.to_excel(r'C:\projetos\Analise-de-cancelamentos-de-matricula-de-universidade\dados\dados-coletados.xlsx', index = None)

print('Programa encerrado!')

Programa encerrado!


In [ ]:
driver.close()